In [14]:
import kagglehub

# Download latest version
dl = kagglehub.dataset_download("pienik/unemployment-in-canada-by-province-1976-present")
path ='/root/.cache/kaggle/input/unemployment-in-canada-by-province-1976-present/Unemployment_Canada_1976_present.csv'
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kaggle/input/unemployment-in-canada-by-province-1976-present/Unemployment_Canada_1976_present.csv


In [17]:
from kagglehub import KaggleDatasetAdapter
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "pienik/unemployment-in-canada-by-province-1976-present",
    "Unemployment_Canada_1976_present.csv"
)

In [8]:
import pandas as pd

In [18]:
df.head()

,REF_DATE,GEO,Sex,Age group,Employment,Full-time employment,Labour force,Part-time employment,Population,Unemployment,Employment rate,Participation rate,Unemployment rate
0,1976-01,Alberta,Both sexes,15 to 24 years,231800.0,174900.0,252300.0,56900.0,362300.0,20500.0,64.0,69.6,8.1
1,1976-01,Alberta,Both sexes,15 to 64 years,802400.0,682100.0,837500.0,120300.0,1154800.0,35000.0,69.5,72.5,4.2
2,1976-01,Alberta,Both sexes,15 years and over,819500.0,693700.0,856500.0,125800.0,1276700.0,37000.0,64.2,67.1,4.3
3,1976-01,Alberta,Both sexes,25 to 54 years,491400.0,439800.0,505800.0,51600.0,661700.0,14400.0,74.3,76.4,2.8
4,1976-01,Alberta,Both sexes,25 years and over,587700.0,518800.0,604200.0,68900.0,914400.0,16500.0,64.3,66.1,2.7


In [24]:
print(df.shape)
df.dtypes

(38985, 13)


,0
REF_DATE,object
GEO,object
Sex,object
Age group,object
Employment,float64
Full-time employment,float64
Labour force,float64
Part-time employment,float64
Population,float64
Unemployment,float64


In [26]:
print(df.isnull().sum())

REF_DATE                    0
GEO                         0
Sex                         0
Age group                   0
Employment                  0
Full-time employment     1695
Labour force                0
Part-time employment     1696
Population                  0
Unemployment                6
Employment rate             0
Participation rate          0
Unemployment rate           6
dtype: int64


In [29]:
obj_cols = df.select_dtypes(include='object').columns
for col in obj_cols:
    uniques = df[col].unique()
    print(f"Column `{col}` – {len(uniques)} unique value(s):")
    print(uniques)
    print("-" * 40)

Column `REF_DATE` – 565 unique value(s):
['1976-01' '1976-02' '1976-03' '1976-04' '1976-05' '1976-06' '1976-07'
 '1976-08' '1976-09' '1976-10' '1976-11' '1976-12' '1977-01' '1977-02'
 '1977-03' '1977-04' '1977-05' '1977-06' '1977-07' '1977-08' '1977-09'
 '1977-10' '1977-11' '1977-12' '1978-01' '1978-02' '1978-03' '1978-04'
 '1978-05' '1978-06' '1978-07' '1978-08' '1978-09' '1978-10' '1978-11'
 '1978-12' '1979-01' '1979-02' '1979-03' '1979-04' '1979-05' '1979-06'
 '1979-07' '1979-08' '1979-09' '1979-10' '1979-11' '1979-12' '1980-01'
 '1980-02' '1980-03' '1980-04' '1980-05' '1980-06' '1980-07' '1980-08'
 '1980-09' '1980-10' '1980-11' '1980-12' '1981-01' '1981-02' '1981-03'
 '1981-04' '1981-05' '1981-06' '1981-07' '1981-08' '1981-09' '1981-10'
 '1981-11' '1981-12' '1982-01' '1982-02' '1982-03' '1982-04' '1982-05'
 '1982-06' '1982-07' '1982-08' '1982-09' '1982-10' '1982-11' '1982-12'
 '1983-01' '1983-02' '1983-03' '1983-04' '1983-05' '1983-06' '1983-07'
 '1983-08' '1983-09' '1983-10' '1983

NameError: name 'null' is not defined